# Préparation des variables de mode de vie

In [2]:
library(tidyverse)
library(factoextra)

# Charger les données
DataPilot <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/01_pilot_2025.rds")

# Charger le fichier des variables
source("01-1_variables.R")

df_mode_de_vie <- DataPilot %>%
  select(all_of(variables_mode_de_vie))

In [3]:
library(dplyr)

df_mode_de_vie %>%
  select(starts_with("lifestyle_typeTransport")) %>%
  summarise(across(everything(), \(x) sum(x, na.rm = TRUE)))
#  summarise(across(everything(), sum, na.rm = TRUE))

lifestyle_typeTransportCar,lifestyle_typeTransportSUV,lifestyle_typeTransportMoto,lifestyle_typeTransportWalk,lifestyle_typeTransportBicycle,lifestyle_typeTransportPublicTransit
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
598,204,6,88,11,111


In [4]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwellingApp 

  0   1 
881 140 


Table pour la variable: ses_dwellingLoft 

   0    1 
1012    9 


Table pour la variable: ses_dwellingCondo 

  0   1 
925  96 


Table pour la variable: ses_dwellingTour 

  0   1 
968  53 


Table pour la variable: ses_dwellingDetachedHouse 

  0   1 
486 535 


Table pour la variable: ses_dwellingTownhouse 

  0   1 
941  80 


Table pour la variable: ses_dwellingDuplex 

  0   1 
968  53 


Table pour la variable: ses_dwellingCoop 

   0    1 
1014    7 


Table pour la variable: ses_dwellingHLM 

   0    1 
1014    7 


Table pour la variable: ses_dwellingMobile 

   0    1 
1011   10 


Table pour la variable: ses_dwellingOther 

  0   1 
990  31 


Table pour la variable: lifestyle_typeTransportCar 

  0   1 
423 598 


Table pour la variable: lifestyle_typeTransportSUV 

  0   1 
817 204 


Table pour la variable: lifestyle_typeTransportMoto 

   0    1 
1015    6 


Table pour la variable: lifestyle_typeTransportWa

### Transformations

- Regroupement de ses_dwelling
  - ses_dwellingCondo : ses_dwellingCondo + ses_dwellingLoft + ses_dwellingTour
  - ses_dwellingOther : ses_dwellingOther + ses_dwellingCoop + ses_dwellingHLM + ses_dwellingMobile

- Regroupement des act_transport
  - lifestyle_typeTransportCar : lifestyle_typeTransportCar + lifestyle_typeTransportMoto (parce qu'il y avraiment trop peu de moto, assigner la classe la plus populeuse)
  - lifestyle_typeTransportActive:   lifestyle_typeTransportWalk + lifestyle_typeTransportBicycle
  - Les autres ne sont pas regroupés

In [5]:
# Regroupement de ses_dwelling
#-----------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    ses_dwellingCondo = if_else(
      ses_dwellingLoft == 1 | ses_dwellingTour == 1,
      1,
      ses_dwellingCondo
    ),
    ses_dwellingOther = if_else(
      ses_dwellingCoop == 1 | ses_dwellingHLM == 1 | ses_dwellingMobile == 1,
      1,
      ses_dwellingOther
    )
  ) %>%
  select(
    -ses_dwellingLoft,
    -ses_dwellingTour,
    -ses_dwellingCoop,
    -ses_dwellingHLM,
    -ses_dwellingMobile
  )

In [6]:
# Regroupement de act_transport
#------------------------------

df_mode_de_vie <- df_mode_de_vie %>%
  mutate(
    lifestyle_typeTransportCar = if_else(
      lifestyle_typeTransportMoto == 1 | lifestyle_typeTransportSUV == 1,
      1,
      lifestyle_typeTransportCar
    ),
    lifestyle_typeTransportNoCar = if_else(
      lifestyle_typeTransportWalk == 1 | lifestyle_typeTransportBicycle == 1 | lifestyle_typeTransportPublicTransit == 1,
      1,
      0
    )
  ) %>%
  select(
    -lifestyle_typeTransportMoto,
    -lifestyle_typeTransportSUV,
    -lifestyle_typeTransportWalk,
    -lifestyle_typeTransportBicycle,
    -lifestyle_typeTransportPublicTransit)

In [7]:
variables_mode_de_vie_clust <- c(
  "id",
  #"postal_code", # À transformer en rural, urbain, région, banlieue
  "ses_dwellingApp",
  "ses_dwellingCondo",
  "ses_dwellingDetachedHouse",
  "ses_dwellingTownhouse",
  "ses_dwellingDuplex",
  "ses_dwellingOther",
  "lifestyle_typeTransportCar",
  "lifestyle_typeTransportNoCar"
)

In [8]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwellingApp 

  0   1 
881 140 


Table pour la variable: ses_dwellingCondo 

  0   1 
863 158 


Table pour la variable: ses_dwellingDetachedHouse 

  0   1 
486 535 


Table pour la variable: ses_dwellingTownhouse 

  0   1 
941  80 


Table pour la variable: ses_dwellingDuplex 

  0   1 
968  53 


Table pour la variable: ses_dwellingOther 

  0   1 
966  55 


Table pour la variable: lifestyle_typeTransportCar 

  0   1 
213 808 


Table pour la variable: lifestyle_typeTransportNoCar 

  0   1 
811 210 




In [9]:
# Sauvegarder les données préparées
saveRDS(df_mode_de_vie, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/02_pilot_2025_mode_de_vie.rds")

---

# SUGGESTION : Activité de Clustering

Si souhaité, effectuer un exercice de clustering avec ce sous-groupe de variables pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.